In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
df=pd.read_csv(r'pcs_data (04182022)_ARASAT.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.set_index('Timestamp')
df.head()

,acAR,acAS,acAT,acPowerFactor,actKw
Timestamp,,,,,
2022-04-14 02:16:00,0.2,0.5,1.6,5.41,0.2
2022-04-14 02:16:00,0.2,0.4,1.5,5.40,0.2
2022-04-14 02:16:00,0.2,0.4,1.5,5.40,0.2
2022-04-14 02:16:00,0.2,0.4,1.6,5.62,0.2
2022-04-14 02:16:00,0.2,0.4,1.6,5.62,0.2


In [4]:
daily_df = df.copy()
print(len(daily_df))
daily_df.head()


256345


,acAR,acAS,acAT,acPowerFactor,actKw
Timestamp,,,,,
2022-04-14 02:16:00,0.2,0.5,1.6,5.41,0.2
2022-04-14 02:16:00,0.2,0.4,1.5,5.40,0.2
2022-04-14 02:16:00,0.2,0.4,1.5,5.40,0.2
2022-04-14 02:16:00,0.2,0.4,1.6,5.62,0.2
2022-04-14 02:16:00,0.2,0.4,1.6,5.62,0.2


In [5]:
train_df,test_df = daily_df[:100000], daily_df[100000:120000] 


In [6]:
print((train_df).shape)

(100000, 5)


In [7]:
train = train_df
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler()
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

<ipython-input-7-538f0eb9a691>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
<ipython-input-7-538f0eb9a691>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
<ipython-input-7-538f0eb9a691>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr

In [8]:
def split_series(series, n_past, n_future):
    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        # slicing the past and future parts of the window
        past, future = series[window_start:past_end, :], series[past_end:future_end, :]
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)

In [9]:
n_past = 120
n_future = 12 
n_features = 5

In [10]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [11]:
print(X_train)

[[[0.00645161 0.01910828 0.02222222 0.19926874 0.99029126]
  [0.00645161 0.01273885 0.01481481 0.19744059 0.99029126]
  [0.00645161 0.01273885 0.01481481 0.19744059 0.99029126]
  ...
  [0.00645161 0.01273885 0.02222222 0.34552102 1.        ]
  [0.00645161 0.01273885 0.02222222 0.34552102 1.        ]
  [0.00645161 0.01910828 0.02222222 0.25045704 1.        ]]

 [[0.00645161 0.01273885 0.01481481 0.19744059 0.99029126]
  [0.00645161 0.01273885 0.01481481 0.19744059 0.99029126]
  [0.00645161 0.01273885 0.02222222 0.23765996 0.99029126]
  ...
  [0.00645161 0.01273885 0.02222222 0.34552102 1.        ]
  [0.00645161 0.01910828 0.02222222 0.25045704 1.        ]
  [0.00645161 0.01910828 0.02222222 0.25045704 1.        ]]

 [[0.00645161 0.01273885 0.01481481 0.19744059 0.99029126]
  [0.00645161 0.01273885 0.02222222 0.23765996 0.99029126]
  [0.00645161 0.01273885 0.02222222 0.23765996 0.99029126]
  ...
  [0.00645161 0.01910828 0.02222222 0.25045704 1.        ]
  [0.00645161 0.01910828 0.0222222

In [12]:
print(y_train)

[[[0.00645161 0.01910828 0.02222222 0.25045704 1.        ]
  [0.00645161 0.01910828 0.02222222 0.2797075  1.        ]
  [0.00645161 0.01910828 0.02222222 0.2797075  1.        ]
  ...
  [0.00645161 0.01273885 0.02222222 0.06581353 0.99029126]
  [0.00645161 0.01273885 0.02222222 0.06581353 0.99029126]
  [0.00645161 0.01273885 0.02222222 0.13711152 0.99029126]]

 [[0.00645161 0.01910828 0.02222222 0.2797075  1.        ]
  [0.00645161 0.01910828 0.02222222 0.2797075  1.        ]
  [0.00645161 0.01910828 0.02222222 0.17184644 0.99029126]
  ...
  [0.00645161 0.01273885 0.02222222 0.06581353 0.99029126]
  [0.00645161 0.01273885 0.02222222 0.13711152 0.99029126]
  [0.00645161 0.01273885 0.02222222 0.13711152 0.99029126]]

 [[0.00645161 0.01910828 0.02222222 0.2797075  1.        ]
  [0.00645161 0.01910828 0.02222222 0.17184644 0.99029126]
  [0.00645161 0.01910828 0.02222222 0.17184644 0.99029126]
  ...
  [0.00645161 0.01273885 0.02222222 0.13711152 0.99029126]
  [0.00645161 0.01273885 0.0222222

In [13]:
print(X_train.shape,y_train.shape)

(99869, 120, 5) (99869, 12, 5)


In [14]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 120, 5)]     0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100), (None, 42400       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 12, 100)      0           lstm[0][0]                       
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 12, 100)      80400       repeat_vector[0][0]              
                                                                 lstm[0][1]                   

In [15]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)


In [16]:
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),batch_size=1024,verbose=1,callbacks=[reduce_lr])


Epoch 1/50
98/98 [==============================] - 14s 51ms/step - loss: 0.0178 - val_loss: 0.1092
Epoch 2/50
98/98 [==============================] - 4s 36ms/step - loss: 0.0010 - val_loss: 0.1086
Epoch 3/50
98/98 [==============================] - 4s 36ms/step - loss: 0.0010 - val_loss: 0.1069
Epoch 4/50
98/98 [==============================] - 4s 37ms/step - loss: 9.5906e-04 - val_loss: 0.1048
Epoch 5/50
98/98 [==============================] - 4s 36ms/step - loss: 9.0438e-04 - val_loss: 0.1036
Epoch 6/50
98/98 [==============================] - 4s 36ms/step - loss: 9.1335e-04 - val_loss: 0.1042
Epoch 7/50
98/98 [==============================] - 4s 37ms/step - loss: 8.9311e-04 - val_loss: 0.1045
Epoch 8/50
98/98 [==============================] - 4s 37ms/step - loss: 8.6986e-04 - val_loss: 0.1056
Epoch 9/50
98/98 [==============================] - 4s 36ms/step - loss: 8.4192e-04 - val_loss: 0.1067
Epoch 10/50
98/98 [==============================] - 4s 36ms/step - loss: 8.4845e-04

In [17]:
pred_e1d = model_e1d1.predict(X_train)

In [18]:
pred_e1d1=model_e1d1.predict(X_test)


In [19]:
for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]
    pred_e1d1[:,:,index]=scaler.inverse_transform(pred_e1d1[:,:,index])
    pred_e1d[:,:,index]=scaler.inverse_transform(pred_e1d[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

In [20]:
from sklearn.metrics import mean_absolute_error
for index,i in enumerate(train_df.columns):
    print(i)
    for j in range(1,12):
        print("Second ",j,":")
        print("MAE-E1D1 : ",mean_absolute_error(y_test[:,j-1,index],pred_e1d1[:,j-1,index]),end=", ")
        print()
        print()

acAR
Second  1 :
MAE-E1D1 :  2.450727430465903, 

Second  2 :
MAE-E1D1 :  2.2917260849959145, 

Second  3 :
MAE-E1D1 :  2.2571124730780436, 

Second  4 :
MAE-E1D1 :  2.265590073136456, 

Second  5 :
MAE-E1D1 :  2.2903297006111285, 

Second  6 :
MAE-E1D1 :  2.3201102306411623, 

Second  7 :
MAE-E1D1 :  2.350144028844865, 

Second  8 :
MAE-E1D1 :  2.3793368956626404, 

Second  9 :
MAE-E1D1 :  2.4062205640739798, 

Second  10 :
MAE-E1D1 :  2.4326233664092385, 

Second  11 :
MAE-E1D1 :  2.4579642389641774, 

acAS
Second  1 :
MAE-E1D1 :  2.532742505520695, 

Second  2 :
MAE-E1D1 :  1.7557850528117878, 

Second  3 :
MAE-E1D1 :  1.4084890560554257, 

Second  4 :
MAE-E1D1 :  1.2970457640484911, 

Second  5 :
MAE-E1D1 :  1.2963625366473654, 

Second  6 :
MAE-E1D1 :  1.3419932425490062, 

Second  7 :
MAE-E1D1 :  1.4026501357410084, 

Second  8 :
MAE-E1D1 :  1.4647591419022028, 

Second  9 :
MAE-E1D1 :  1.521665351150132, 

Second  10 :
MAE-E1D1 :  1.5727396192871617, 

Second  11 :
MAE-E1D1 :  1

In [21]:
print(pred_e1d[3])

[[0.28323436 0.50922316 1.5771701  5.684572   0.32740572]
 [0.22600959 0.46352074 1.5858479  5.660538   0.30341467]
 [0.19849221 0.45136163 1.5718144  5.620744   0.27784273]
 [0.18446353 0.441842   1.5614862  5.5812674  0.26570845]
 [0.17842886 0.43373132 1.5567164  5.5453806  0.26084185]
 [0.1806529  0.4294005  1.5576824  5.514364   0.26138395]
 [0.185334   0.42792365 1.5617541  5.488297   0.26324597]
 [0.19302095 0.42768228 1.5664172  5.465827   0.2629998 ]
 [0.19988015 0.42877963 1.5703212  5.446638   0.26234904]
 [0.20712288 0.4296503  1.5744382  5.4307227  0.2610248 ]
 [0.21263123 0.43027207 1.5755866  5.416107   0.25908354]
 [0.21552528 0.4322112  1.5725702  5.402287   0.25510776]]


In [22]:
print(y_train[0])

[[0.2  0.5  1.6  5.69 0.3 ]
 [0.2  0.5  1.6  5.85 0.3 ]
 [0.2  0.5  1.6  5.85 0.3 ]
 [0.2  0.5  1.6  5.26 0.2 ]
 [0.2  0.5  1.6  5.26 0.2 ]
 [0.2  0.4  1.6  5.33 0.2 ]
 [0.2  0.4  1.6  5.33 0.2 ]
 [0.2  0.4  1.5  4.72 0.2 ]
 [0.2  0.4  1.5  4.72 0.2 ]
 [0.2  0.4  1.6  4.68 0.2 ]
 [0.2  0.4  1.6  4.68 0.2 ]
 [0.2  0.4  1.6  5.07 0.2 ]]


In [23]:
model_e1d1.save('mmLSTM_ARASAT.h5')